# Octave Convolution Kod İncelemesi + Örnek Model (OctNet)

Bu notebook’ta iki şeyi birlikte yapacağız:

1) **Octave Convolution (OctConv)** kod yapısını *satır satır mantığıyla* inceleyeceğiz.
2) OctConv kullanan **tam bir mini model** (backbone + head) yazacağız ve çalıştıracağız.

Notebook içinde:
- OctConv’un 4 yolunu (H→H, H→L, L→H, L→L) kodda nerede yaptığımızı net göstereceğim.
- `alpha` (α) kanal bölme mantığını kod üzerinde takip edeceğiz.
- Forward sırasında **tensor şekillerini** (shape) print ederek akışı gözleyeceğiz.

Not: Bu notebook eğitim amaçlıdır; performans optimizasyonu değil, **anlaşılır tasarım** önceliklidir.


## 1) OctConv’un Temel Fikri (1 paragraf)

Feature map kanallarının bir kısmı **High** bantta (tam çözünürlük H×W), bir kısmı **Low** bantta (genelde H/2×W/2) tutulur.
OctConv, girişte `(x_h, x_l)` alıp çıkışta `(y_h, y_l)` üretir.
İçeride 4 akış vardır:

- **H→H**: High’dan High’a (aynı çözünürlük)
- **H→L**: High’dan Low’a (downsample + conv)
- **L→L**: Low’dan Low’a (düşük çözünürlükte conv)
- **L→H**: Low’dan High’a (conv + upsample)


## 2) Kullanacağımız Yardımcı Operasyonlar

OctConv’u basitçe kurmak için:
- **Downsample:** `AvgPool2d(2)`
- **Upsample:** `F.interpolate(scale_factor=2, mode='nearest')`

Bu iki operasyon, High ve Low bantlar arasında geçişi sağlar.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

## 3) OctaveConv2d (Çekirdek Katman) — Kod İncelemesi

Bu sınıf OctConv’un tam kendisi.

### 3.1) `__init__` içinde yapılanlar

1) `alpha_in`, `alpha_out` ile kanal bölme (High/Low) miktarı belirlenir.
2) `cin_h, cin_l, cout_h, cout_l` hesaplanır.
3) 4 akış için conv tanımlanır:
   - `hh`: H→H
   - `hl`: H→L
   - `ll`: L→L
   - `lh`: L→H

### 3.2) `forward` içinde yapılanlar

Her akışın çıktısı toplanarak `y_h` ve `y_l` oluşturulur.
Yani kodun mantığı şu iki satıra indirgenir:

- `y_h = HH(x_h) + Up(LH(x_l))`
- `y_l = LL(x_l) + HL(Down(x_h))`


In [2]:
class OctaveConv2d(nn.Module):
    """Octave Convolution (Ara katman versiyonu)

    Girdi:
        x_h: (B, C_h, H, W)
        x_l: (B, C_l, H/2, W/2) veya None

    Çıktı:
        y_h: (B, C_out_h, H_out, W_out) veya None
        y_l: (B, C_out_l, H_out/2, W_out/2) veya None

    Not:
        Bu implementasyon eğitim amaçlı: down=AvgPool2d, up=nearest.
    """
    def __init__(self, cin, cout, kernel_size=3, stride=1, padding=1,
                 alpha_in=0.5, alpha_out=0.5, bias=False):
        super().__init__()

        assert 0.0 <= alpha_in <= 1.0
        assert 0.0 <= alpha_out <= 1.0

        self.alpha_in = alpha_in
        self.alpha_out = alpha_out
        self.stride = stride

        # 1) Kanal bölme
        cin_l = int(round(cin * alpha_in))
        cin_h = cin - cin_l
        cout_l = int(round(cout * alpha_out))
        cout_h = cout - cout_l

        self.cin_h, self.cin_l = cin_h, cin_l
        self.cout_h, self.cout_l = cout_h, cout_l

        # 2) Bant geçişleri için downsample
        self.down = nn.AvgPool2d(kernel_size=2, stride=2)

        # 3) 4 akış için conv'lar
        # Kanal sayısı 0 ise ilgili yol kullanılmaz (None)
        self.hh = nn.Conv2d(cin_h, cout_h, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_h > 0 and cout_h > 0) else None
        self.hl = nn.Conv2d(cin_h, cout_l, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_h > 0 and cout_l > 0) else None
        self.ll = nn.Conv2d(cin_l, cout_l, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_l > 0 and cout_l > 0) else None
        self.lh = nn.Conv2d(cin_l, cout_h, kernel_size, stride=stride, padding=padding, bias=bias) if (cin_l > 0 and cout_h > 0) else None

    def forward(self, x_h, x_l=None):
        # y_h ve y_l'yi toplama değişkeni olarak başlatıyoruz.
        # int başlatıp en sonda None'a çevirmemizin sebebi: bazı yollarda kanal=0 olabilir.
        y_h = 0
        y_l = 0

        # --- H -> H ---
        if self.hh is not None and x_h is not None:
            y_h = y_h + self.hh(x_h)

        # --- H -> L --- (downsample + conv)
        if self.hl is not None and x_h is not None:
            x_h_down = self.down(x_h)          # (B, C_h, H/2, W/2)
            y_l = y_l + self.hl(x_h_down)

        # --- L -> L ---
        if self.ll is not None and x_l is not None:
            y_l = y_l + self.ll(x_l)

        # --- L -> H --- (conv + upsample)
        if self.lh is not None and x_l is not None:
            y_lh = self.lh(x_l)                # (B, C_out_h, H/2, W/2)
            y_lh_up = F.interpolate(y_lh, scale_factor=2, mode='nearest')  # (B, C_out_h, H, W)
            y_h = y_h + y_lh_up

        # Eğer ilgili bant yoksa (0 kanal seçildiyse), y_* int kalır → None yap.
        if isinstance(y_h, int):
            y_h = None
        if isinstance(y_l, int):
            y_l = None

        return y_h, y_l


## 4) OctaveConvBlock — OctConv + BN + ReLU

Backbone’da pratik kullanım için OctConv’un çıkışlarına BN + aktivasyon ekleriz.
High ve Low bantların kanal sayıları farklı olacağı için BN’leri ayrı tutuyoruz.


In [3]:
class OctaveConvBlock(nn.Module):
    def __init__(self, cin, cout, alpha_in=0.5, alpha_out=0.5,
                 kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.oct = OctaveConv2d(cin, cout, kernel_size, stride, padding, alpha_in, alpha_out, bias=False)

        self.bn_h = nn.BatchNorm2d(self.oct.cout_h) if self.oct.cout_h > 0 else None
        self.bn_l = nn.BatchNorm2d(self.oct.cout_l) if self.oct.cout_l > 0 else None
        self.act = nn.ReLU(inplace=True)

    def forward(self, x_h, x_l=None):
        y_h, y_l = self.oct(x_h, x_l)
        if y_h is not None and self.bn_h is not None:
            y_h = self.act(self.bn_h(y_h))
        if y_l is not None and self.bn_l is not None:
            y_l = self.act(self.bn_l(y_l))
        return y_h, y_l


## 5) Split / Merge Fonksiyonları (Tek Bant ↔ Çift Bant)

OctConv kullanırken tipik akış:

1) Stem ile tek bant feature üretirsin: `x` (B, C, H, W)
2) Bunu `(x_h, x_l)` diye iki banda ayırırsın (split)
3) Birkaç OctConv blok uygularsın
4) Çıkışta low bandı upsample edip high ile concat yaparsın (merge)

Aşağıdaki iki fonksiyon bu işi yapar.


In [4]:
class SplitMergeHL(nn.Module):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.down = nn.AvgPool2d(2)

    def split(self, x):
        """x: (B, C, H, W) -> (x_h, x_l)"""
        B, C, H, W = x.shape
        c_l = int(round(C * self.alpha))
        c_h = C - c_l
        x_h = x[:, :c_h]
        x_l = self.down(x[:, c_h:]) if c_l > 0 else None
        return x_h, x_l

    def merge(self, x_h, x_l):
        """(x_h, x_l) -> (B, C, H, W)"""
        if x_l is None:
            return x_h
        x_l_up = F.interpolate(x_l, scale_factor=2, mode='nearest')
        return torch.cat([x_h, x_l_up], dim=1)


## 6) OctConv Kullanan Tam Model: `OctNetMini`

Bu modelin mimarisi:

1) **Stem:** `Conv(3→64)` + BN + ReLU
2) **Split:** 64 kanalı (α ile) High/Low’a ayır
3) **Backbone:** 2 adet `OctaveConvBlock`
4) **Merge:** Low’u upsample edip High ile concat → tek tensöre dön
5) **Head:** GlobalAvgPool + Linear

Bu mimari “OctConv’u modele nasıl entegre ederim?” sorusunun minimal ama doğru cevabıdır.


In [5]:
class OctNetMini(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.sm = SplitMergeHL(alpha=alpha)

        # 1) Stem
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # 2) OctConv backbone
        self.b1 = OctaveConvBlock(64, 128, alpha_in=alpha, alpha_out=alpha)
        self.b2 = OctaveConvBlock(128, 128, alpha_in=alpha, alpha_out=alpha)

        # 3) Head
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x, verbose=False):
        x = self.stem(x)  # (B,64,H,W)
        if verbose:
            print('Stem out:', x.shape)

        x_h, x_l = self.sm.split(x)
        if verbose:
            print('Split x_h:', None if x_h is None else x_h.shape)
            print('Split x_l:', None if x_l is None else x_l.shape)

        x_h, x_l = self.b1(x_h, x_l)
        if verbose:
            print('After b1 x_h:', None if x_h is None else x_h.shape)
            print('After b1 x_l:', None if x_l is None else x_l.shape)

        x_h, x_l = self.b2(x_h, x_l)
        if verbose:
            print('After b2 x_h:', None if x_h is None else x_h.shape)
            print('After b2 x_l:', None if x_l is None else x_l.shape)

        x = self.sm.merge(x_h, x_l)
        if verbose:
            print('Merged x:', x.shape)

        x = self.pool(x)
        x = torch.flatten(x, 1)
        logits = self.fc(x)
        if verbose:
            print('Logits:', logits.shape)
        return logits


## 7) Modeli Çalıştır: Shape Takibi

Aşağıdaki hücrede `verbose=True` verip forward akışında tüm shape’leri göreceğiz.
Bu, kod incelemesinin en net kısmı: **H/Low bantların nasıl aktığını gözle görmek.**


In [6]:
model = OctNetMini(in_channels=3, num_classes=10, alpha=0.5)
x = torch.randn(2, 3, 32, 32)
out = model(x, verbose=True)

print('\nFinal output:', out.shape)


Stem out: torch.Size([2, 64, 32, 32])
Split x_h: torch.Size([2, 32, 32, 32])
Split x_l: torch.Size([2, 32, 16, 16])
After b1 x_h: torch.Size([2, 64, 32, 32])
After b1 x_l: torch.Size([2, 64, 16, 16])
After b2 x_h: torch.Size([2, 64, 32, 32])
After b2 x_l: torch.Size([2, 64, 16, 16])
Merged x: torch.Size([2, 128, 32, 32])
Logits: torch.Size([2, 10])

Final output: torch.Size([2, 10])


## 8) Parametre Sayımı (Opsiyonel Kontrol)

OctConv’un asıl kazancı FLOP tarafındadır; ama yine de toplam parametreyi görmek faydalıdır.


In [7]:
def count_params(m: nn.Module):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

print('Trainable params:', count_params(model))


Trainable params: 224842


## 9) Kod İnceleme Notları (Kritik Noktalar)

1) **`alpha` kanal bölme yapar**
- Stem çıkışı 64 kanal.
- `alpha=0.5` ise: Low ≈ 32 kanal, High ≈ 32 kanal.
- Low bant ayrıca `AvgPool2d(2)` ile **H/2×W/2** çözünürlüğe iner.

2) **H→L yolu neden downsample ister?**
- Low bant zaten düşük çözünürlükte tutuluyor.
- High’tan Low’a bilgi aktarırken çözünürlüğü eşitlemek zorundasın.

3) **L→H yolu neden upsample ister?**
- Low’tan High’a aktarırken High çözünürlüğe geri çıkmalısın.

4) **Toplama (fusion) nerede oluyor?**
- High çıkış: `y_h = HH(x_h) + Up(LH(x_l))`
- Low çıkış: `y_l = LL(x_l) + HL(Down(x_h))`

5) **Merge neden concat?**
- En sonda klasik katmanlara dönmek için tek tensör gerekir.
- High (H×W) ile upsample edilmiş Low (H×W) kanalda birleştirilir.
